In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
df_train = pd.read_csv("/kaggle/input/playground-series-s5e4/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s5e4/test.csv")
df_submission = pd.read_csv("/kaggle/input/playground-series-s5e4/sample_submission.csv")

In [4]:
df_train.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [5]:
df_train.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,750000.000000,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,374999.500000,64.504738,59.859901,52.236449,1.348855,45.437406
std,216506.495284,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000
25%,187499.750000,35.730000,39.410000,28.380000,0.000000,23.178350
50%,374999.500000,63.840000,60.050000,53.580000,1.000000,43.379460
75%,562499.250000,94.070000,79.530000,76.600000,2.000000,64.811580
max,749999.000000,325.240000,119.460000,119.910000,103.910000,119.970000


In [6]:
df_train.isnull().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

## Train test Split

In [7]:
X = df_train.copy()
y = df_train["Listening_Time_minutes"]

In [8]:
from sklearn.model_selection import train_test_split
X_train_initial,X_test_initial, y_train_initial, y_test_initial = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train_initial.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
453635,453635,True Crime Stories,Episode 81,94.30,True Crime,97.67,Friday,Afternoon,NaN,2.0,Positive,77.27788
11651,11651,Learning Lab,Episode 53,82.15,Education,94.78,Saturday,Night,NaN,1.0,Positive,50.02839
431999,431999,Tech Trends,Episode 21,13.72,Technology,68.60,Saturday,Afternoon,65.77,3.0,Negative,10.07496
529211,529211,Laugh Line,Episode 99,24.00,Comedy,42.14,Saturday,Morning,41.29,0.0,Negative,17.82074
110925,110925,Athlete's Arena,Episode 15,NaN,Sports,34.10,Friday,Night,NaN,0.0,Neutral,94.80341


## Fix percentage outliers

In [10]:
X_train_initial[X_train_initial["Host_Popularity_percentage"] > 100 ].shape

(22, 12)

In [11]:
Guest_median_train = X_train_initial["Guest_Popularity_percentage"].median()
Guest_median_test = X_test_initial["Guest_Popularity_percentage"].median()
#Guest_median_sub = df_test["Guest_Popularity_percentage"].median()

Host_median_train = X_train_initial["Host_Popularity_percentage"].median()
Host_median_test = X_test_initial["Host_Popularity_percentage"].median()
#Host_median_sub = df_test["Host_Popularity_percentage"].median()

In [12]:
X_train_initial["Guest_Popularity_percentage"] = X_train_initial.apply(lambda row:row["Guest_Popularity_percentage"] if row["Guest_Popularity_percentage"] <= 100 else Guest_median_train, axis=1)
X_test_initial["Guest_Popularity_percentage"] = X_test_initial.apply(lambda row:row["Guest_Popularity_percentage"] if row["Guest_Popularity_percentage"] <= 100 else Guest_median_test, axis=1)
#df_test["Guest_Popularity_percentage"] = df_test.apply(lambda row:row["Guest_Popularity_percentage"] if row["Guest_Popularity_percentage"] <= 100 else Guest_median_sub, axis=1)


In [13]:
X_train_initial["Host_Popularity_percentage"] = X_train_initial.apply(lambda row:row["Host_Popularity_percentage"] if row["Host_Popularity_percentage"] <= 100 else Host_median_train, axis=1)
X_test_initial["Host_Popularity_percentage"] = X_test_initial.apply(lambda row:row["Host_Popularity_percentage"] if row["Host_Popularity_percentage"] <= 100 else Host_median_test, axis=1)
#df_test["Host_Popularity_percentage"] = df_test.apply(lambda row:row["Host_Popularity_percentage"] if row["Host_Popularity_percentage"] <= 100 else Host_median_sub, axis=1)


## Fix Number of Ads

In [14]:
X_train_initial[X_train_initial["Number_of_Ads"] > 4 ].shape

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


(4, 12)

In [15]:
ads_median_train = X_train_initial["Number_of_Ads"].median()
ads_median_test = X_test_initial["Number_of_Ads"].median()
#ads_median_sub = df_test["Number_of_Ads"].median()

In [16]:
X_train_initial["Number_of_Ads"] = X_train_initial.apply(lambda row:row["Number_of_Ads"] if row["Number_of_Ads"] <= 4 else ads_median_train, axis=1)
X_test_initial["Number_of_Ads"] = X_test_initial.apply(lambda row:row["Number_of_Ads"] if row["Number_of_Ads"] <= 4 else ads_median_test, axis=1)
#df_test["Number_of_Ads"] = df_test.apply(lambda row:row["Number_of_Ads"] if row["Number_of_Ads"] <= 4 else ads_median_sub, axis=1)


## Fix Episode Length

In [17]:
X_train_initial["Episode_Length_minutes"] = X_train_initial.apply(lambda row: row["Episode_Length_minutes"] if row["Episode_Length_minutes"] > row["Listening_Time_minutes"] else row["Listening_Time_minutes"], axis=1)
X_test_initial["Episode_Length_minutes"] = X_test_initial.apply(lambda row: row["Episode_Length_minutes"] if row["Episode_Length_minutes"] > row["Listening_Time_minutes"] else row["Listening_Time_minutes"], axis=1)

In [18]:
test_length_median = df_test["Episode_Length_minutes"].median()

In [19]:
#df_test[df_test["Episode_Length_minutes"] > 7575.0].head()

In [20]:
#df_test.loc[54434, "Episode_Length_minutes"] = test_length_median

In [21]:
#df_test.loc[56597, "Episode_Length_minutes"] = test_length_median

In [22]:
#df_test.loc[54434, "Episode_Length_minutes"]

In [23]:
df_test.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads
count,250000.000000,2.212640e+05,250000.000000,201168.000000,250000.000000
mean,874999.500000,4.192987e+02,59.716491,52.192796,1.355852
std,72168.927986,1.668545e+05,22.880028,28.445034,4.274399
min,750000.000000,2.470000e+00,2.490000,0.000000,0.000000
25%,812499.750000,3.578000e+01,39.250000,28.320000,0.000000
50%,874999.500000,6.397000e+01,59.900000,53.360000,1.000000
75%,937499.250000,9.415000e+01,79.390000,76.560000,2.000000
max,999999.000000,7.848626e+07,117.760000,116.820000,2063.000000


In [24]:
df_test.isnull().sum()

id                                 0
Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         28736
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    48832
Number_of_Ads                      0
Episode_Sentiment                  0
dtype: int64

In [25]:
y_test_initial.head()

404846    20.91979
580313    42.66054
552086    43.30069
370876    48.56000
239330    48.53842
Name: Listening_Time_minutes, dtype: float64

na values in Episode_Length_minutes in df_test are not filled

## Encoding

In [26]:
X_train_initial.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
453635,453635,True Crime Stories,Episode 81,94.30000,True Crime,97.67,Friday,Afternoon,53.62,2.0,Positive,77.27788
11651,11651,Learning Lab,Episode 53,82.15000,Education,94.78,Saturday,Night,53.62,1.0,Positive,50.02839
431999,431999,Tech Trends,Episode 21,13.72000,Technology,68.60,Saturday,Afternoon,65.77,3.0,Negative,10.07496
529211,529211,Laugh Line,Episode 99,24.00000,Comedy,42.14,Saturday,Morning,41.29,0.0,Negative,17.82074
110925,110925,Athlete's Arena,Episode 15,94.80341,Sports,34.10,Friday,Night,53.62,0.0,Neutral,94.80341


In [27]:
X_train_dropped = X_train_initial.drop(columns = ['id', 'Listening_Time_minutes'], axis = 1)
X_test_dropped = X_test_initial.drop(columns = ['id', 'Listening_Time_minutes'], axis =1)

In [28]:
X_train_dropped.head()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
453635,True Crime Stories,Episode 81,94.30000,True Crime,97.67,Friday,Afternoon,53.62,2.0,Positive
11651,Learning Lab,Episode 53,82.15000,Education,94.78,Saturday,Night,53.62,1.0,Positive
431999,Tech Trends,Episode 21,13.72000,Technology,68.60,Saturday,Afternoon,65.77,3.0,Negative
529211,Laugh Line,Episode 99,24.00000,Comedy,42.14,Saturday,Morning,41.29,0.0,Negative
110925,Athlete's Arena,Episode 15,94.80341,Sports,34.10,Friday,Night,53.62,0.0,Neutral


In [29]:
X_train_encoded = X_train_dropped.copy()
X_test_encoded = X_test_dropped.copy()

from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
categorical_columns = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
for col in categorical_columns:
    X_train_encoded[col] = label_encoder.fit_transform(X_train_encoded[col])
    X_test_encoded[col] = label_encoder.transform(X_test_encoded[col])
    df_test[col] = label_encoder.transform(df_test[col])

In [30]:
df_test.set_index('id', inplace=True)

In [31]:
df_test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
id,,,,,,,,,,
750000,11,71,78.96,2,38.11,2,1,53.33,1.0,1
750001,36,16,27.87,5,71.29,3,2,NaN,0.0,1
750002,24,3,69.10,1,67.89,0,1,97.51,0.0,2
750003,4,71,115.39,1,23.40,3,2,51.75,2.0,2
750004,27,46,72.32,4,58.10,6,2,11.30,2.0,1


## Model XGBoost

In [32]:
import xgboost as xgb
from xgboost import XGBRegressor

model_xgb = XGBRegressor(objective='reg:squarederror', n_estimators=18000, learning_rate=0.05, max_depth=7, random_state=42, device="cuda", early_stopping_rounds=100)
model_xgb.fit(X_train_encoded, y_train_initial, eval_set=[(X_train_encoded, y_train_initial), (X_test_encoded, y_test_initial)], verbose=100)

[0]	validation_0-rmse:26.01063	validation_1-rmse:25.99620
[100]	validation_0-rmse:10.76903	validation_1-rmse:10.82527
[200]	validation_0-rmse:10.64446	validation_1-rmse:10.78020
[300]	validation_0-rmse:10.55298	validation_1-rmse:10.75646
[400]	validation_0-rmse:10.46661	validation_1-rmse:10.74010
[500]	validation_0-rmse:10.39137	validation_1-rmse:10.72647
[600]	validation_0-rmse:10.32100	validation_1-rmse:10.71534
[700]	validation_0-rmse:10.25238	validation_1-rmse:10.70468
[800]	validation_0-rmse:10.19437	validation_1-rmse:10.69687
[900]	validation_0-rmse:10.13189	validation_1-rmse:10.69092
[1000]	validation_0-rmse:10.07512	validation_1-rmse:10.68423
[1100]	validation_0-rmse:10.01643	validation_1-rmse:10.67854
[1200]	validation_0-rmse:9.96127	validation_1-rmse:10.67364
[1300]	validation_0-rmse:9.90624	validation_1-rmse:10.66937
[1400]	validation_0-rmse:9.85396	validation_1-rmse:10.66633
[1500]	validation_0-rmse:9.80953	validation_1-rmse:10.66307
[1600]	validation_0-rmse:9.76338	validat

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=100,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=18000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [33]:
#df_test.to_device("cuda")
df_submission['Listening_Time_minutes'] = model_xgb.predict(df_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [15:09:39] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [34]:
df_submission.head()

,id,Listening_Time_minutes
0,750000,56.981491
1,750001,23.646561
2,750002,50.227337
3,750003,73.622498
4,750004,47.594067


In [35]:
df_submission.to_csv('submission10.762.csv', index=False)
